# Playground RAG ChatBot

load data
https://python.langchain.com/docs/integrations/providers/unstructured
https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.directory.DirectoryLoader.html

In [ ]:
#import langchain
import chromadb
import os
import shutil

from openai import OpenAI

from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, SystemMessage


In [ ]:
DATA_PATH = "../data/"

def load_docs():
    loader = DirectoryLoader(DATA_PATH, glob="*", show_progress=True)
    documents = loader.load()
    return documents

docs = load_docs()
docs

split document in to chunks
https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=True,
)

chunks = text_splitter.split_documents(docs)

print(f"Len docs: {len(docs)}")
print(f"Len chunks: {len(chunks)}")

In [ ]:
doc = chunks[0]
doc.metadata

Querie with ChromaDB
https://www.trychroma.com/
https://api.python.langchain.com/en/latest/embeddings/langchain_community.embeddings.openai.OpenAIEmbeddings.html

load openaI api key
https://platform.openai.com/api-keys

In [ ]:
file_path = 'YourOpenAiKey.txt'
try:
    with open(file_path, 'r') as file:
        key = file.read()
        result = f"Key: {key}"
except FileNotFoundError:
    result = "Die Datei wurde nicht gefunden."

result

In [ ]:
CHROMA_PATH = "../chroma"

if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

chromadb = Chroma.from_documents(
    chunks, OpenAIEmbeddings(openai_api_key=key), persist_directory=CHROMA_PATH
)
chromadb.persist()

predict
https://api.python.langchain.com/en/v0.0.343/embeddings/langchain.embeddings.openai.OpenAIEmbeddings.html

In [ ]:
embedding_function = OpenAIEmbeddings(openai_api_key=key)
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

In [ ]:
query_text = "Wer ist Tobias Meißner?" #user input

In [ ]:
results = db.similarity_search_with_score(query_text, k=3)
# filter result
# if len(results) == 0 or results[0][1] < 0.7:
#     print(f"Unable to find matching results.")

In [ ]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
context_text

In [ ]:
# create prompt
client = OpenAI(api_key=key)
completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": f"Answer the question based only on the following context: {context_text}"},
        {"role": "user", "content": f"{query_text}"}
    ],
)

print(completion.choices[0].message.content)


In [ ]:
completion

In [ ]:
    #output
    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {completion.choices[0].message.content}\nSources: {sources}"
    print(formatted_response)